# [The Battle of Neighborhoods](https://www.coursera.org/learn/applied-data-science-capstone/home/week/4)
 - **Wenzhuo Song**
 - **wenzhuosong1996@outlook.com**

### 1. A description of the problem and a discussion of the background.

<font size=3.5>We have learned to analyze data and get data sets by Foursquare API before, and in the final capstone project, we need to analyze an open end task with learned skills. The task is, we have explored New York City and City of Toronto and clustered their own neighborhoods respectively, and now are interested in how similar or dissimilar they are by comparing their neighborhoods. What is more, we also want to know if New York City is more like City of Toronto or Paris or some other multicultural cities and where we should open a restaurant or start business, which is open and depending on our own thoughts.  
    
<font size=3.5>New York City (often called simply New York) is the most populous city in the United States, which is the center of the New York metropolitan area, the largest metropolitan area in the world by urban landmass, and it has been described as the cultural, financial, and media capital of the world ([Wikipedia](https://en.wikipedia.org/wiki/New_York_City)). Toronto is the capital city of the Canadian province of Ontario, which is the most populous city in Canada and the fourth most populous city in North America and recognized as one of the most multicultural and cosmopolitan cities in the world ([Wikipedia](https://en.wikipedia.org/wiki/Toronto)).   
    
<font size=3.5>Both of New York City and City of Toronto are internationally famous, so they have many things in common, like various foods and sports, but they may also have their own features because of different geography, countries and cultures. Therefore, with analysis of kinds and numbers of venues of neighborhoods in each city, their own features can be found, which can help us analyze the similarity and dissimilarity and make decisions.

### 2. A description of the data and how it will be used to solve the problem. 

<font size=3.5>Because we have learned how to get and analyze neighborhoods of NYC and Toronto, so I will use the same way to get the number (not proportion) of venues of neighborhoods in each city, which I will use to find the key features and then calculate the similarity and dissimilarity between two cities.

#### 2.1 Scrape the data.
##### 2.1.1 NYC

In [1]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [2]:
CLIENT_ID = '1QNBWZXCKW43RHFLX4GYWWZXCF2TBX423SI5E0L3WFDXUJPH' # your Foursquare ID
CLIENT_SECRET = 'NAZMCD34B1AXIWR3BAH1GCHDZF5VGCG3JFN1AEMWMKJ3Y3L5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1QNBWZXCKW43RHFLX4GYWWZXCF2TBX423SI5E0L3WFDXUJPH
CLIENT_SECRET:NAZMCD34B1AXIWR3BAH1GCHDZF5VGCG3JFN1AEMWMKJ3Y3L5


In [3]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        try:
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            nearby_venues.columns = ['Neighborhood', 
                          'Neighborhood Latitude', 
                          'Neighborhood Longitude', 
                          'Venue', 
                          'Venue Latitude', 
                          'Venue Longitude', 
                          'Venue Category']
        except:
            continue
    
    return(nearby_venues)

In [10]:
NY_venues = getNearbyVenues(neighborhoods['Neighborhood'], neighborhoods['Latitude'], neighborhoods['Longitude'])

In [11]:
NY_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [13]:
print('There are {} uniques categories.'.format(len(NY_venues['Venue Category'].unique())))

There are 430 uniques categories.


In [36]:
# one hot encoding
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot['Neighbourhood'] = NY_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,...,Warehouse Store,Waste Facility,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
NY_grouped = NY_onehot.groupby('Neighbourhood').sum().reset_index()
NY_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,...,Warehouse Store,Waste Facility,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Allerton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Annadale,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Arden Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arlington,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Arrochar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
NY_grouped.shape

(300, 431)

In [44]:
# save the NYC data
NY_grouped.to_csv('NV_grouped.csv', index=False)

##### 2.1.2 Totonto

In [45]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df_clean = df.drop(df[df['Borough'].str.contains('Not assigned')].index).reset_index(drop=True)

In [46]:
# define a function to update the neighbourhood
def update_neighbourhood(row):
    if row['Neighbourhood'] == 'Not assigned':
        return row['Borough']
    else:
        return row['Neighbourhood']
df_clean['Neighbourhood'] = df_clean.apply(update_neighbourhood, axis=1)

In [47]:
# read the csv file
geospatial_coordinates = pd.read_csv('D:/Code/Python/Coursera Capstone/Coursera_Capstone/Geospatial_Coordinates.csv')
# merge the dataframe
df_final = pd.merge(df_clean, geospatial_coordinates, on=['Postal Code'])
# change name of the first column
df_final.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [49]:
toronto_data = df_final
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [50]:
neighbourhood_lists = []
for i in toronto_data.Neighbourhood:
    neighbourhood_lists += i.split(', ')

In [52]:
neighbourhood_latitude = []
neighbourhood_longitude = []
delete_list = []
for neighbourhood in neighbourhood_lists:
    address = '{}, Toronto, Ontario'.format(neighbourhood)
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    try:
        neighbourhood_latitude.append(location.latitude)
        neighbourhood_longitude.append(location.longitude)
    except:
        delete_list.append(neighbourhood)
        
# # delete the duplicates
# neighbourhood_lists = list(neighbourhood_lists - set(delete_list))
# toronto_neighbourhood = pd.DataFrame({'Neighbourhood': neighbourhood_lists,
#                                       'Latitude': neighbourhood_latitude, 
#                                       'Longitude': neighbourhood_longitude})
# toronto_neighbourhood.head()

In [56]:
neighbourhood_lists = [temp for temp in neighbourhood_lists if temp not in delete_list]
toronto_neighbourhood = pd.DataFrame({'Neighbourhood': neighbourhood_lists,
                                      'Latitude': neighbourhood_latitude, 
                                      'Longitude': neighbourhood_longitude})
toronto_neighbourhood.head()

,Neighbourhood,Latitude,Longitude
0,Parkwoods,43.758800,-79.320197
1,Victoria Village,43.732658,-79.311189
2,Regent Park,43.660706,-79.360457
3,Harbourfront,43.640080,-79.380150
4,Lawrence Manor,43.722079,-79.437507


In [57]:
toronto_venues = getNearbyVenues(toronto_neighbourhood['Neighbourhood'], 
                                 toronto_neighbourhood['Latitude'], 
                                 toronto_neighbourhood['Longitude'])

In [58]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7588,-79.320197,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.7588,-79.320197,LCBO,43.757774,-79.314257,Liquor Store
2,Parkwoods,43.7588,-79.320197,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy
3,Parkwoods,43.7588,-79.320197,Petro-Canada,43.757950,-79.315187,Gas Station
4,Parkwoods,43.7588,-79.320197,TD Canada Trust,43.757569,-79.314976,Bank


In [59]:
# ont hot encoding
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'], prefix='', prefix_sep='')

# add neighbourhood column back to the dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood']

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Service,American Restaurant,Animal Shelter,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
toronto_grouped = toronto_onehot.groupby(['Neighbourhood']).sum().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Service,American Restaurant,Animal Shelter,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0,0,0,0,0,0,4,0,0,...,1,0,0,0,0,0,1,0,0,0
1,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Agincourt North,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,Albion Gardens,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Alderwood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
toronto_grouped.shape

(194, 336)

In [65]:
# save the Toronto data
toronto_grouped.to_csv('Toronto_grouped.csv', index=False)

##### 2.2 Future Work

<font size=3.5>Now the data of neighborhoods of each city is available, in the next week, the main work is how to calculate two cities' similarity, which includes Feature extraction, Embedding, Clustering and Topic Model.